
<h1>Tranformers<h1> 

In [7]:
!pip install torch torchvision torchaudio --timeout 120


  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp312-cp312-win_amd64.whl.metadata (6.2 kB)
  Using cached torchaudio-2.5.1-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.0 MB 1.3 MB/s eta 0:02:38
   ---------------------------------------- 0.8/203.0 MB 1.5 MB/s eta 0:02:19
   ---------------------------------------- 1.3/203.0 MB 1.8 MB/s eta 0:01:55
   ---------------------------------------- 1.6/203.0 MB 1.8 MB/s et

In [68]:
import torch  # Importa la librería principal de PyTorch, que es utilizada para construir y entrenar modelos de redes neuronales, y realizar operaciones tensoriales.

import torch.nn as nn  # Importa el submódulo `nn` de PyTorch, que contiene clases para construir redes neuronales, como capas (e.g., Linear, Conv2d) y modelos predefinidos.

import torch.nn.functional as F  # Importa el submódulo `functional` de PyTorch, que contiene funciones útiles para redes neuronales, como activaciones (ReLU, Sigmoid) y funciones de pérdida (CrossEntropy, MSELoss).

import torch.optim as optim  # Importa el submódulo `optim` de PyTorch, que proporciona varios optimizadores (como SGD, Adam) para actualizar los parámetros del modelo durante el entrenamiento.

from torch.utils.data import Dataset, DataLoader  # Importa herramientas esenciales para trabajar con datos en PyTorch:
# - `Dataset`: Es la clase base para crear conjuntos de datos personalizados, donde defines cómo acceder a los datos (implementando los métodos `__getitem__` y `__len__`).
# - `DataLoader`: Es utilizado para cargar datos en lotes (batches) durante el entrenamiento de redes neuronales, facilitando el procesamiento eficiente de grandes volúmenes de datos.

from collections import Counter  # Importa la clase `Counter` de la librería `collections`, que se utiliza para contar la frecuencia de elementos en una lista o conjunto de datos, útil en análisis de texto o para datos en general.

import math  # Importa la librería estándar `math` que proporciona funciones matemáticas avanzadas como logaritmos, trigonometría, constantes matemáticas (por ejemplo, `pi`), etc.

import numpy as np  # Importa la librería `numpy` para realizar operaciones eficientes con arrays multidimensionales y álgebra lineal.

import re  # Importa la librería `re` para trabajar con expresiones regulares, que se utiliza para encontrar patrones y realizar búsquedas en texto.

torch.manual_seed(23)  # Establece una semilla para la generación de números aleatorios en PyTorch, con el valor `23`. Esto asegura que los resultados sean reproducibles cada vez que se ejecute el código, garantizando consistencia en la inicialización de pesos y en el muestreo aleatorio.


In [70]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Asigna el dispositivo de computación en el que se ejecutará el modelo. Si hay una GPU disponible (a través de CUDA), se utilizará la GPU ('cuda'). 
#Si no hay GPU disponible, se utilizará la CPU ('cpu').

#print(device)  # Imprime el dispositivo que se ha asignado, es decir, 'cuda' si la GPU está disponible o 'cpu' si solo se puede usar la CPU.

device = torch.device("cpu")  # Asegúrate de usar la CPU
model = model.to(device)  # Mueve el modelo a la CPU
print(device)


cpu


In [71]:
MAX_SEQ_LEN = 50  # Define la longitud máxima de una secuencia de entrada (en este caso, 128 elementos).


In [72]:
class PositionalEmbedding(nn.Module):
    """
    La clase `PositionalEmbedding` genera una matriz de codificación posicional 
    que se añade a los embeddings de las palabras. Esta codificación posicional 
    permite que el modelo Transformer capture información sobre el orden de los tokens en la secuencia.
    """
    def __init__(self, d_model, max_seq_len = MAX_SEQ_LEN):
        """
        Inicializa la codificación posicional.
        Parámetros:
        - d_model: La dimensionalidad del embedding de cada token (también usado en el Transformer).
        - max_seq_len: La longitud máxima de la secuencia de entrada (por defecto, MAX_SEQ_LEN).
        """
        super().__init__()
        
        # Crear la matriz de embeddings posicionales, con tamaño (max_seq_len, d_model)
        self.pos_embed_matrix = torch.zeros(max_seq_len, d_model, device=device)
        
        # Generar las posiciones de los tokens, un tensor de tamaño (max_seq_len, 1)
        token_pos = torch.arange(0, max_seq_len, dtype = torch.float).unsqueeze(1)
        
        # Calcular el término de división para cada dimensión del embedding
        div_term = torch.exp(torch.arange(0, d_model, 2).float() 
                             * (-math.log(10000.0)/d_model))
        
        # Aplicar las funciones seno y coseno a las posiciones
        self.pos_embed_matrix[:, 0::2] = torch.sin(token_pos * div_term)
        self.pos_embed_matrix[:, 1::2] = torch.cos(token_pos * div_term)
        
        # Reorganizar la matriz para la suma posterior con el embedding de entrada
        self.pos_embed_matrix = self.pos_embed_matrix.unsqueeze(0).transpose(0,1)

    def forward(self, x):
        """
        Realiza la operación de suma entre el embedding de entrada y la codificación posicional.
        Parámetros:
        - x: El tensor de entrada con forma (seq_len, batch_size, d_model).
        """
        return x + self.pos_embed_matrix[:x.size(0), :]

    
class MultiHeadAttention(nn.Module):
    """
    La clase `MultiHeadAttention` implementa la atención multi-cabeza del Transformer. 
    La atención multi-cabeza permite que el modelo enfoque en diferentes partes de la secuencia 
    de entrada de forma simultánea.
    """
    def __init__(self, d_model = 512, num_heads = 8):
        """
        Inicializa la atención multi-cabeza.
        Parámetros:
        - d_model: Dimensionalidad del embedding de cada token.
        - num_heads: Número de cabezas de atención.
        """
        super().__init__()
        
        # Asegura que el tamaño del modelo sea divisible por el número de cabezas
        assert d_model % num_heads == 0, 'Embedding size not compatible with num heads'
        
        # Dimensiones de la representación de cada cabeza
        self.d_v = d_model // num_heads
        self.d_k = self.d_v
        self.num_heads = num_heads
        
        # Definición de las matrices de transformación para Q (consulta), K (clave) y V (valor)
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, Q, K, V, mask = None):
        """
        Realiza la operación de atención multi-cabeza.
        Parámetros:
        - Q: Tensor de consultas con forma [batch_size, seq_len, d_model].
        - K: Tensor de claves con forma [batch_size, seq_len, d_model].
        - V: Tensor de valores con forma [batch_size, seq_len, d_model].
        - mask: Máscara para evitar que ciertos tokens se vean durante la atención.
        """
        batch_size = Q.size(0)
        
        # Aplicar las transformaciones lineales a Q, K y V y luego dividirlas en varias cabezas
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # Aplicar la atención de producto punto escalado
        weighted_values, attention = self.scale_dot_product(Q, K, V, mask)
        
        # Reorganizar la salida de las cabezas de atención
        weighted_values = weighted_values.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        
        # Aplicar la transformación final
        weighted_values = self.W_o(weighted_values)
        
        return weighted_values, attention
        
    def scale_dot_product(self, Q, K, V, mask = None):
        """
        Realiza el producto punto escalado entre las consultas y las claves, 
        aplicando la máscara si es necesario.
        Parámetros:
        - Q: Consultas.
        - K: Claves.
        - V: Valores.
        - mask: Máscara para aplicar.
        """
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)  # Aplica la máscara
        attention = F.softmax(scores, dim=-1)  # Aplica softmax
        weighted_values = torch.matmul(attention, V)  # Calcula el valor ponderado
        
        return weighted_values, attention


class PositionFeedForward(nn.Module):
    """
    La clase `PositionFeedForward` es una red feed-forward completamente conectada 
    que se aplica de forma independiente a cada posición de la secuencia.
    """
    def __init__(self, d_model, d_ff):
        """
        Inicializa la red feed-forward.
        Parámetros:
        - d_model: Dimensionalidad del embedding de cada token.
        - d_ff: Dimensionalidad de la capa interna de la red feed-forward.
        """
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)  # Capa lineal de entrada
        self.linear2 = nn.Linear(d_ff, d_model)  # Capa lineal de salida

    def forward(self, x):
        """
        Realiza el paso hacia adelante de la red feed-forward.
        Parámetros:
        - x: Tensor de entrada con forma [batch_size, seq_len, d_model].
        """
        return self.linear2(F.relu(self.linear1(x)))  # Activación ReLU


class EncoderSubLayer(nn.Module):
    """
    La clase `EncoderSubLayer` define una subcapa del encoder del Transformer, 
    que consiste en una capa de atención seguida de una red feed-forward.
    """
    def __init__(self, d_model, num_heads, d_ff, dropout = 0.1):
        """
        Inicializa la subcapa del encoder.
        Parámetros:
        - d_model: Dimensionalidad del embedding de cada token.
        - num_heads: Número de cabezas de atención.
        - d_ff: Dimensionalidad de la red feed-forward.
        - dropout: Tasa de dropout para regularización.
        """
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)  # Atención multi-cabeza
        self.ffn = PositionFeedForward(d_model, d_ff)  # Red feed-forward
        self.norm1 = nn.LayerNorm(d_model)  # Normalización de capa
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)  # Dropout para regularización
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask = None):
        """
        Realiza el paso hacia adelante de la subcapa del encoder.
        Parámetros:
        - x: Tensor de entrada.
        - mask: Máscara para la atención.
        """
        attention_score, _ = self.self_attn(x, x, x, mask)  # Atención de la entrada
        x = x + self.dropout1(attention_score)  # Residual y dropout
        x = self.norm1(x)  # Normalización
        x = x + self.dropout2(self.ffn(x))  # Red feed-forward y residual
        return self.norm2(x)  # Normalización final


class Encoder(nn.Module):
    """
    La clase `Encoder` contiene varias capas de `EncoderSubLayer`, 
    cada una de las cuales realiza atención y procesamiento feed-forward.
    """
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        """
        Inicializa el encoder.
        Parámetros:
        - d_model: Dimensionalidad del embedding de cada token.
        - num_heads: Número de cabezas de atención.
        - d_ff: Dimensionalidad de la red feed-forward.
        - num_layers: Número de capas en el encoder.
        - dropout: Tasa de dropout.
        """
        super().__init__()
        self.layers = nn.ModuleList([EncoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)  # Normalización final del encoder

    def forward(self, x, mask=None):
        """
        Realiza el paso hacia adelante del encoder.
        Parámetros:
        - x: Tensor de entrada.
        - mask: Máscara para la atención.
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


class DecoderSubLayer(nn.Module):
    """
    La clase `DecoderSubLayer` define una subcapa del decoder, que incluye
    atención self-attention, cross-attention (atención entre encoder y decoder),
    y una red feed-forward.
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        Inicializa la subcapa del decoder.
        Parámetros:
        - d_model: Dimensionalidad del embedding de cada token.
        - num_heads: Número de cabezas de atención.
        - d_ff: Dimensionalidad de la red feed-forward.
        - dropout: Tasa de dropout.
        """
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)  # Atención self
        self.cross_attn = MultiHeadAttention(d_model, num_heads)  # Atención entre encoder y decoder
        self.feed_forward = PositionFeedForward(d_model, d_ff)  # Red feed-forward
        self.norm1 = nn.LayerNorm(d_model)  # Normalización
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)  # Dropout
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, encoder_output, target_mask=None, encoder_mask=None):
        """
        Realiza el paso hacia adelante de la subcapa del decoder.
        Parámetros:
        - x: Tensor de entrada (target).
        - encoder_output: Salida del encoder (la que el decoder usa como contexto).
        - target_mask: Máscara para la atención del target.
        - encoder_mask: Máscara para la atención del encoder.
        """
        attention_score, _ = self.self_attn(x, x, x, target_mask)  # Self-attention
        x = x + self.dropout1(attention_score)
        x = self.norm1(x)
        
        encoder_attn, _ = self.cross_attn(x, encoder_output, encoder_output, encoder_mask)  # Cross-attention
        x = x + self.dropout2(encoder_attn)
        x = self.norm2(x)
        
        ff_output = self.feed_forward(x)  # Red feed-forward
        x = x + self.dropout3(ff_output)
        return self.norm3(x)


class Decoder(nn.Module):
    """
    La clase `Decoder` contiene varias capas de `DecoderSubLayer`, 
    cada una de las cuales realiza atención self-attention, cross-attention 
    y procesamiento feed-forward.
    """
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        """
        Inicializa el decoder.
        Parámetros:
        - d_model: Dimensionalidad del embedding de cada token.
        - num_heads: Número de cabezas de atención.
        - d_ff: Dimensionalidad de la red feed-forward.
        - num_layers: Número de capas en el decoder.
        - dropout: Tasa de dropout.
        """
        super().__init__()
        self.layers = nn.ModuleList([DecoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)  # Normalización final del decoder

    def forward(self, x, encoder_output, target_mask, encoder_mask):
        """
        Realiza el paso hacia adelante del decoder.
        Parámetros:
        - x: Tensor de entrada (target).
        - encoder_output: Salida del encoder.
        - target_mask: Máscara para la atención del target.
        - encoder_mask: Máscara para la atención del encoder.
        """
        for layer in self.layers:
            x = layer(x, encoder_output, target_mask, encoder_mask)
        return self.norm(x)


In [73]:
class Transformer(nn.Module):  
    # Definición de la clase Transformer, que hereda de nn.Module para construir un modelo de red neuronal.
    # Esta clase representa la arquitectura completa del Transformer: un modelo de atención con encoder-decoder.
    
    def __init__(self, d_model, num_heads, d_ff, num_layers,
                 input_vocab_size, target_vocab_size, 
                 max_len=MAX_SEQ_LEN, dropout=0.1):
        # El método constructor inicializa los componentes del Transformer.
        # Parámetros:
        # - d_model: Dimensionalidad de los vectores de características.
        # - num_heads: Número de cabezas de atención en cada capa de atención multi-cabeza.
        # - d_ff: Dimensionalidad de la red completamente conectada en el modelo.
        # - num_layers: Número de capas en el encoder y decoder.
        # - input_vocab_size: Tamaño del vocabulario de entrada.
        # - target_vocab_size: Tamaño del vocabulario de salida (target).
        # - max_len: Longitud máxima de las secuencias de entrada y salida.
        # - dropout: Tasa de dropout para regularización durante el entrenamiento.
        
        super().__init__()  # Llama al constructor de la clase base nn.Module.
        
        # Definición de las capas del modelo.
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)  
        # Capa de embedding para la entrada (vocabulario de entrada -> d_model).
        
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        # Capa de embedding para la salida (vocabulario de salida -> d_model).
        
        self.pos_embedding = PositionalEmbedding(d_model, max_len)
        # Capa para codificar las posiciones de los tokens en las secuencias de entrada y salida.
        
        self.encoder = Encoder(d_model, num_heads, d_ff, num_layers, dropout)
        # Definición del encoder con el número de capas (num_layers), atención multi-cabeza (num_heads) y otros parámetros.
        
        self.decoder = Decoder(d_model, num_heads, d_ff, num_layers, dropout)
        # Definición del decoder con los mismos parámetros que el encoder.
        
        self.output_layer = nn.Linear(d_model, target_vocab_size)
        # Capa lineal que convierte la salida del decoder a las predicciones finales (tamaño vocabulario de salida).
        
    def forward(self, source, target):
        # Método `forward` define cómo se realiza el paso hacia adelante a través del modelo.
        # Parámetros:
        # - source: Secuencia de entrada (source).
        # - target: Secuencia de salida esperada (target).
        
        # Generación de las máscaras para el encoder y decoder.
        source_mask, target_mask = self.mask(source, target)
        
        # Embedding y codificación posicional para la entrada (source).
        source = self.encoder_embedding(source) * math.sqrt(self.encoder_embedding.embedding_dim)
        # Realiza el embedding de la entrada y escala por la raíz cuadrada de la dimensión del embedding.
        
        source = self.pos_embedding(source)  
        # Añade la codificación posicional a la entrada.
        
        # Paso del encoder.
        encoder_output = self.encoder(source, source_mask)
        
        # Embedding y codificación posicional para la salida (target).
        target = self.decoder_embedding(target) * math.sqrt(self.decoder_embedding.embedding_dim)
        # Realiza el embedding de la salida y escala de manera similar.
        
        target = self.pos_embedding(target)  
        # Añade la codificación posicional a la salida.
        
        # Paso del decoder.
        output = self.decoder(target, encoder_output, target_mask, source_mask)
        # El decoder recibe la salida del encoder y las máscaras correspondientes para generar la salida.
        
        return self.output_layer(output)
        # La salida del decoder se pasa por una capa lineal que produce la predicción final para cada token en la secuencia de salida.

    
    def mask(self, source, target):
        # Método para crear las máscaras para el encoder y decoder.
        # Las máscaras se utilizan para evitar que el modelo vea tokens futuros durante el entrenamiento (en el caso del decoder).
        
        source_mask = (source != 0).unsqueeze(1).unsqueeze(2)  
        # Máscara para la entrada (source): crea una máscara para los tokens no nulos (diferentes de 0).
        
        target_mask = (target != 0).unsqueeze(1).unsqueeze(2)  
        # Máscara para la salida (target): crea una máscara similar para la secuencia de salida.
        
        size = target.size(1)  
        # Obtiene el tamaño de la secuencia de salida.
        
        no_mask = torch.tril(torch.ones((1, size, size), device=device)).bool()  
        # Crea una máscara triangular inferior para el decoder, asegurándose de que cada posición solo pueda "ver" las posiciones anteriores (para evitar mirar futuros tokens).
        
        target_mask = target_mask & no_mask
        # Aplica la máscara triangular inferior a la máscara de la salida (target_mask).
        
        return source_mask, target_mask  
        # Devuelve las máscaras para la entrada y la salida.


In [74]:
# Definición de parámetros de entrada y salida para el modelo
seq_len_source = 10  # Longitud de la secuencia de entrada (source).
seq_len_target = 10  # Longitud de la secuencia de salida (target).
batch_size = 2  # Número de ejemplos en cada lote (batch).
input_vocab_size = 50  # Tamaño del vocabulario para la secuencia de entrada (source). 50
target_vocab_size = 50  # Tamaño del vocabulario para la secuencia de salida (target). 50

# Generación de las secuencias de entrada (source) y salida (target)
# Las secuencias de entrada y salida son vectores de enteros que representan índices
# de palabras en los respectivos vocabularios.
# `torch.randint` genera números enteros aleatorios en el rango [1, vocab_size).
source = torch.randint(1, input_vocab_size, (batch_size, seq_len_source))  # Secuencias de entrada (source)
target = torch.randint(1, target_vocab_size, (batch_size, seq_len_target))  # Secuencias de salida (target)


In [75]:
# Definición de hiperparámetros para el modelo Transformer
d_model = 512 # Dimensionalidad de los vectores de características (embeddings) de entrada y salida.  #512
               # Este valor define el tamaño de la representación interna de cada token.
num_heads = 8  # Número de cabezas en la atención multi-cabeza (multi-head attention).
               # Esto permite al modelo aprender diferentes representaciones para cada token en distintas subespacios de características.
d_ff = 2048  # Tamaño de la capa de alimentación directa (feed-forward layer) interna. #2048
             # Es el número de neuronas en la capa oculta de la red de alimentación directa.
num_layers = 6  # Número de capas en el encoder y decoder del Transformer.
               # A mayor número de capas, mayor capacidad de modelado, pero también mayor complejidad computacional.

# Inicialización del modelo Transformer
# Se crea una instancia del modelo Transformer con los hiperparámetros definidos previamente.
# Este modelo es capaz de procesar secuencias de entrada y salida usando un mecanismo de atención
# multi-cabeza y una red de alimentación directa.
model = Transformer(d_model, num_heads, d_ff, num_layers,
                  input_vocab_size, target_vocab_size, 
                  max_len=MAX_SEQ_LEN, dropout=0.1)

# Mover el modelo al dispositivo adecuado (CPU o GPU)
# Esto garantiza que el modelo se ejecute en el dispositivo donde la variable `device` haya sido configurada (puede ser 'cuda' si hay GPU disponible o 'cpu' si no).
model = model.to(device)

# Mover las secuencias de entrada (source) y salida (target) al mismo dispositivo que el modelo.
# Esto asegura que las secuencias de entrada y salida estén en el dispositivo correcto para el procesamiento
# durante la etapa de entrenamiento o inferencia.
source = source.to(device)  # Mueve las secuencias de entrada (source) al dispositivo
target = target.to(device)  # Mueve las secuencias de salida (target) al dispositivo


In [76]:
output = model(source, target)  # Realiza una pasada hacia adelante a través del modelo Transformer, generando las predicciones de salida para cada token en la secuencia 'target' basada en las secuencias de entrada 'source'.


In [77]:
# Expected output shape -> [batch, seq_len_target, target_vocab_size] i.e. [2, 10, 50]
print(f'output.shape {output.shape}')  # Imprime las dimensiones del tensor de salida, mostrando la forma de la salida del modelo (batch_size, seq_len_target, target_vocab_size).


output.shape torch.Size([2, 10, 50])


<h1>Translation Eng-Span</h1>

In [78]:
import pandas as pd

# Definimos la ruta del archivo CSV que contiene las frases en inglés y español
PATH = 'ing-esp.csv'  # Asegúrate de que la ruta sea correcta

# Intentamos leer el archivo CSV especificando el delimitador correcto (punto y coma ';')
# Leemos solo las columnas que nos interesan (1 para inglés y 3 para español)
df = pd.read_csv(PATH, sep=';', encoding='ISO-8859-1', header=None, dtype=str, low_memory=False, on_bad_lines='skip')

# Verificamos las primeras filas para entender la estructura del DataFrame
print(df.head())



     0                                                 1        2   \
0  1276                              Let's try something.     2481   
1  1277                            I have to go to sleep.     2482   
2  1280  Today is June 18th and it is Muiriel's birthday!     2485   
3  1280  Today is June 18th and it is Muiriel's birthday!  1130137   
4  1282                                Muiriel is 20 now.     2487   

                                                  3    4    5    6    7    8   \
0                                  ¡Intentemos algo!  NaN  NaN  NaN  NaN  NaN   
1                           Tengo que irme a dormir.  NaN  NaN  NaN  NaN  NaN   
2  ¡Hoy es 18 de junio y es el cumpleaños de Muir...  NaN  NaN  NaN  NaN  NaN   
3  ¡Hoy es el 18 de junio y es el cumpleaños de M...  NaN  NaN  NaN  NaN  NaN   
4                      Ahora, Muiriel tiene 20 años.  NaN  NaN  NaN  NaN  NaN   

    9    10  
0  NaN  NaN  
1  NaN  NaN  
2  NaN  NaN  
3  NaN  NaN  
4  NaN  NaN  


In [79]:
# Seleccionamos solo las columnas 1 (inglés) y 3 (español)
eng_spa_cols = df.iloc[:, [1, 3]]

# Eliminamos cualquier columna vacía (si existiera alguna columna extra)
eng_spa_cols = eng_spa_cols.dropna(axis=1, how='all')

# Calculamos la longitud de las frases en inglés
eng_spa_cols['length'] = eng_spa_cols.iloc[:, 0].str.len()

# Ordenamos por la longitud de las frases en inglés
eng_spa_cols = eng_spa_cols.sort_values(by='length')

# Eliminamos la columna 'length' ya que no es necesaria para el archivo final
eng_spa_cols = eng_spa_cols.drop(columns=['length'])

# Especificamos la ruta de salida para el archivo
output_file_path = 'textoIngleEspaniol.txt'

# Guardamos el DataFrame resultante en un archivo de texto (tabulado y sin índice ni encabezados)
eng_spa_cols.to_csv(output_file_path, sep='\t', index=False, header=False)

In [80]:
PATH = 'textoIngleEspaniol.txt'

In [81]:
# Abrimos el archivo en modo de lectura ('r') y con codificación 'utf-8' para manejar correctamente los caracteres especiales.
# La variable 'PATH' debe contener la ruta del archivo que estamos leyendo.
with open(PATH, 'r', encoding='utf-8') as f:
    # Leemos todas las líneas del archivo y las almacenamos en una lista llamada 'lines'.
    # Cada elemento de la lista 'lines' será una cadena de texto que corresponde a una línea del archivo.
    lines = f.readlines()  # `readlines()` lee todo el contenido del archivo y lo divide por líneas.

# Ahora procesamos cada línea en 'lines', y por cada línea:
# 1. Eliminamos cualquier espacio en blanco o salto de línea extra con 'strip()'.
# 2. Dividimos la línea en dos partes usando el delimitador de tabulación ('\t') con 'split()'.
#    Esto creará una lista con dos elementos: la primera frase (en inglés) y la segunda (en español).
# 3. Filtramos las líneas que no contienen el delimitador '\t', asegurándonos de que solo procesamos las líneas con pares de frases.
eng_spa_pairs = [line.strip().split('\t') for line in lines if '\t' in line]

# 'eng_spa_pairs' será ahora una lista de listas donde cada sublista contiene dos elementos:
# 1. La primera posición es la frase en inglés.
# 2. La segunda posición es la traducción en español.
# Esto se utilizará para tareas como entrenamiento de un modelo de traducción, donde cada par de frases (inglés, español) es un ejemplo de entrenamiento.


In [82]:
# Ahora que hemos procesado las líneas y tenemos 'eng_spa_pairs', vamos a tomar los primeros 10 pares de frases
# para verificar que el procesamiento de los datos se haya realizado correctamente.
# Esto es útil para inspeccionar una pequeña muestra de los datos antes de utilizarla para entrenar un modelo.
eng_spa_pairs[:10]

[['So?', '¿Y?'],
 ['OK.', '¡Órale!'],
 ['Hi!', '¡Hola!'],
 ['Go.', 'Ve.'],
 ['Hi.', '¡Hola!'],
 ['No.', 'No.'],
 ['Go.', 'Vete.'],
 ['Ah!', '¡Anda!'],
 ['Go.', 'Váyase.'],
 ['No!', '¡No!']]

In [83]:
# Primero, verificamos que cada par de frases tiene exactamente dos elementos
# Para evitar que se generen errores de índice fuera de rango
eng_spa_pairs = [pair for pair in eng_spa_pairs if len(pair) == 2]

# Ahora podemos proceder a extraer las frases en inglés y español sin problema
eng_sentences = [pair[0] for pair in eng_spa_pairs]  # Frases en inglés (primer elemento de cada par)
spa_sentences = [pair[1] for pair in eng_spa_pairs]  # Frases en español (segundo elemento de cada par)

# Verificamos los primeros 10 elementos de las listas para asegurarnos de que todo esté correcto
print(eng_sentences[:10])
print(spa_sentences[:10])



['So?', 'OK.', 'Hi!', 'Go.', 'Hi.', 'No.', 'Go.', 'Ah!', 'Go.', 'No!']
['¿Y?', '¡Órale!', '¡Hola!', 'Ve.', '¡Hola!', 'No.', 'Vete.', '¡Anda!', 'Váyase.', '¡No!']


In [84]:
def preprocess_sentence(sentence):
    # 1. Convertir todo el texto a minúsculas y eliminar espacios al principio y al final.
    sentence = sentence.lower().strip()
    
    # 2. Reemplazar múltiples espacios seguidos por un solo espacio.
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    # 3. Normalizar las vocales con tilde a su forma sin tilde (por ejemplo, "á" a "a").
    sentence = re.sub(r"[á]+", "a", sentence)
    sentence = re.sub(r"[é]+", "e", sentence)
    sentence = re.sub(r"[í]+", "i", sentence)
    sentence = re.sub(r"[ó]+", "o", sentence)
    sentence = re.sub(r"[ú]+", "u", sentence)
    
    # 4. Eliminar todos los caracteres que no son letras del alfabeto (como signos de puntuación, números, etc.)
    sentence = re.sub(r"[^a-z]+", " ", sentence)
    
    # 5. Volver a eliminar cualquier espacio extra al principio o al final.
    sentence = sentence.strip()
    
    # 6. Añadir las etiquetas <sos> al principio y <eos> al final de la frase.
    sentence = '<sos> ' + sentence + ' <eos>'
    
    # 7. Devolver la frase procesada.
    return sentence


In [85]:
s1 = '¿Hola @ cómo estás? 123'

In [86]:
# Imprime la variable 's1' en su forma original.
# La variable 's1' contiene una cadena de texto (string) que puede tener diferentes caracteres especiales, espacios extras, o incluso signos de puntuación.
# Al utilizar 'print(s1)', estamos visualizando cómo es la cadena antes de cualquier tipo de preprocesamiento.
print(s1)

# Aplica la función 'preprocess_sentence' a la cadena 's1' y luego imprime el resultado.
# La función 'preprocess_sentence' toma la cadena 's1', la procesa para convertirla a minúsculas, 
# eliminar caracteres no alfabéticos, eliminar espacios innecesarios, normalizar las vocales con tildes, y agregar etiquetas especiales al inicio y fin.
# El resultado es una versión más "limpia" de la cadena original, que es más adecuada para tareas de procesamiento de lenguaje natural.
# Se imprime la cadena procesada para que se pueda comparar con la forma original de 's1'.
print(preprocess_sentence(s1))


¿Hola @ cómo estás? 123
<sos> hola como estas <eos>


In [87]:
# Aplica la función 'preprocess_sentence' a cada una de las frases en la lista 'eng_sentences'.
# La lista 'eng_sentences' contiene frases en inglés, y queremos preprocesar cada una de esas frases antes de usarlas en un modelo de procesamiento de lenguaje.
# Esto asegura que todas las frases en inglés sean limpiadas, normalizadas y formateadas de manera consistente (por ejemplo, convirtiéndolas a minúsculas, eliminando caracteres especiales y agregando las etiquetas <sos> y <eos>).
# Usamos una lista por comprensión para aplicar la función 'preprocess_sentence' a cada frase de 'eng_sentences'.
eng_sentences = [preprocess_sentence(sentence) for sentence in eng_sentences]

# Aplica la misma función 'preprocess_sentence' a cada una de las frases en la lista 'spa_sentences'.
# La lista 'spa_sentences' contiene frases en español, y de igual forma, necesitamos preprocesarlas para hacerlas aptas para su uso en el modelo.
# Al igual que en el caso de las frases en inglés, se aplica la función de preprocesamiento a cada frase en 'spa_sentences' para limpiarlas y estructurarlas correctamente.
spa_sentences = [preprocess_sentence(sentence) for sentence in spa_sentences]


In [88]:
spa_sentences[:10]

['<sos> y <eos>',
 '<sos> orale <eos>',
 '<sos> hola <eos>',
 '<sos> ve <eos>',
 '<sos> hola <eos>',
 '<sos> no <eos>',
 '<sos> vete <eos>',
 '<sos> anda <eos>',
 '<sos> vayase <eos>',
 '<sos> no <eos>']

In [89]:
# Definición de la función 'build_vocab' que se encarga de construir el vocabulario
# a partir de una lista de oraciones. El vocabulario es esencial para convertir
# las palabras en sus correspondientes índices (y viceversa), lo cual es necesario
# para trabajar con modelos de aprendizaje automático.

def build_vocab(sentences):
    # 'sentences' es una lista de oraciones (frases) que puede contener texto en inglés, español
    # o cualquier otro idioma, dependiendo del contexto de uso de la función.
    
    # 1. Se extraen todas las palabras de todas las oraciones, de manera que cada palabra
    #    se convierte en un único elemento en una lista. Para ello, primero recorremos cada
    #    oración, luego separamos cada oración en palabras usando 'split()', y agregamos
    #    todas las palabras a la lista 'words'.
    words = [word for sentence in sentences for word in sentence.split()]
    
    # 2. Usamos un objeto 'Counter' para contar cuántas veces aparece cada palabra en la lista 'words'.
    #    'word_count' es un diccionario que contiene como claves las palabras, y como valores,
    #    la cantidad de veces que cada palabra aparece en todas las oraciones.
    word_count = Counter(words)
    
    # 3. Ordenamos el diccionario de frecuencias 'word_count' de mayor a menor, es decir,
    #    las palabras más frecuentes aparecerán primero en la lista ordenada.
    #    'sorted_word_counts' es una lista de tuplas donde el primer elemento es la palabra
    #    y el segundo es el conteo de ocurrencias de esa palabra.
    sorted_word_counts = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
    
    # 4. Creamos el diccionario 'word2idx' que mapea cada palabra a un índice único.
    #    El índice comienza en 2 porque los índices 0 y 1 se reservan para los tokens especiales '<pad>' y '<unk>'.
    #    Usamos 'enumerate' para asignar un índice a cada palabra ordenada, comenzando desde el índice 2.
    word2idx = {word: idx for idx, (word, _) in enumerate(sorted_word_counts, 2)}
    
    # 5. Asignamos los índices 0 y 1 a los tokens especiales:
    #    - '<pad>': Se usa para rellenar secuencias a un tamaño constante (padding).
    #    - '<unk>': Se usa para representar palabras desconocidas (out-of-vocabulary).
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1
    
    # 6. Creamos un diccionario 'idx2word' que es el inverso de 'word2idx', es decir,
    #    mapea índices a palabras. Esto nos permite recuperar una palabra a partir de su índice.
    idx2word = {idx: word for word, idx in word2idx.items()}
    
    # 7. Finalmente, la función devuelve dos diccionarios:
    #    - 'word2idx': Mapea palabras a índices (para usar en el modelo).
    #    - 'idx2word': Mapea índices a palabras (útil para la interpretación de resultados).
    return word2idx, idx2word


In [90]:
# Construimos los vocabularios para las frases en inglés y español usando la función 'build_vocab'.
# 'build_vocab' toma como entrada una lista de oraciones y devuelve dos diccionarios:
# - 'word2idx': Mapea las palabras a índices numéricos.
# - 'idx2word': Mapea los índices numéricos a palabras.
# En este caso, estamos construyendo los vocabularios para las frases en inglés ('eng_sentences') y español ('spa_sentences').

# Construcción del vocabulario para las frases en inglés.
eng_word2idx, eng_idx2word = build_vocab(eng_sentences)
# - 'eng_word2idx' será un diccionario que asigna un índice único a cada palabra en las frases en inglés.
# - 'eng_idx2word' será el diccionario inverso, que permite obtener la palabra a partir de su índice.

# Construcción del vocabulario para las frases en español.
spa_word2idx, spa_idx2word = build_vocab(spa_sentences)
# - 'spa_word2idx' será un diccionario similar al anterior, pero para las frases en español.
# - 'spa_idx2word' será el diccionario inverso de 'spa_word2idx', para convertir índices en palabras.

# Determinamos el tamaño del vocabulario de inglés y español (número de palabras únicas en cada idioma).
eng_vocab_size = len(eng_word2idx)
# - 'eng_vocab_size' almacenará el número total de palabras únicas (incluyendo los tokens especiales <pad>, <unk>) en el vocabulario de inglés.

spa_vocab_size = len(spa_word2idx)
# - 'spa_vocab_size' almacenará el número total de palabras únicas (incluyendo los tokens especiales <pad>, <unk>) en el vocabulario de español.



In [91]:
print(eng_vocab_size, spa_vocab_size)

27464 46617


In [92]:
# Definimos la clase 'EngSpaDataset' que hereda de 'Dataset'.
# Esta clase se utiliza para crear un conjunto de datos personalizado (dataset) en PyTorch.
# El objetivo de esta clase es proporcionar una forma estructurada de acceder a las frases en inglés y español
# y convertir las palabras en índices numéricos usando los vocabularios previamente construidos.

class EngSpaDataset(Dataset):
    # El constructor (__init__) se encarga de inicializar el conjunto de datos.
    def __init__(self, eng_sentences, spa_sentences, eng_word2idx, spa_word2idx):
        # 'eng_sentences' son las frases en inglés.
        # 'spa_sentences' son las frases en español.
        # 'eng_word2idx' es el diccionario que mapea las palabras en inglés a índices.
        # 'spa_word2idx' es el diccionario que mapea las palabras en español a índices.
        self.eng_sentences = eng_sentences  # Almacenamos las frases en inglés.
        self.spa_sentences = spa_sentences  # Almacenamos las frases en español.
        self.eng_word2idx = eng_word2idx  # Almacenamos el vocabulario de inglés.
        self.spa_word2idx = spa_word2idx  # Almacenamos el vocabulario de español.
        
    # El método __len__ devuelve el tamaño del conjunto de datos.
    # En este caso, el número de oraciones en inglés (y en español, que son iguales).
    def __len__(self):
        # Devuelve la cantidad de frases en el conjunto de datos.
        return len(self.eng_sentences)
    
    # El método __getitem__ obtiene un par de frases (inglés, español) dado un índice 'idx'.
    # Este método será utilizado por el DataLoader de PyTorch para acceder a los datos de manera eficiente.
    def __getitem__(self, idx):
        # 'eng_sentence' es la oración en inglés en el índice 'idx'.
        eng_sentence = self.eng_sentences[idx]
        # 'spa_sentence' es la oración en español en el índice 'idx'.
        spa_sentence = self.spa_sentences[idx]
        
        # Convertimos la oración en inglés en una lista de índices, donde cada palabra se reemplaza por su índice en el vocabulario.
        # Si una palabra no se encuentra en el vocabulario, se utiliza el índice del token desconocido (<unk>).
        eng_idxs = [self.eng_word2idx.get(word, self.eng_word2idx['<unk>']) for word in eng_sentence.split()]
        
        # Convertimos la oración en español en una lista de índices, de manera similar a lo hecho con el inglés.
        # Si una palabra no se encuentra en el vocabulario, se utiliza el índice del token desconocido (<unk>).
        spa_idxs = [self.spa_word2idx.get(word, self.spa_word2idx['<unk>']) for word in spa_sentence.split()]
        
        # Retornamos las oraciones convertidas en índices como tensores de PyTorch.
        # Los tensores son estructuras de datos que PyTorch puede procesar.
        return torch.tensor(eng_idxs), torch.tensor(spa_idxs)


In [93]:
# Definimos la función 'collate_fn', que es utilizada para procesar un lote de datos en el DataLoader.
# Esta función se encarga de manejar el procesamiento de datos dentro del lote antes de ser alimentados al modelo.
# Principalmente, realiza dos tareas importantes: 
# - Recortar o limitar las secuencias a una longitud máxima.
# - Realizar padding (relleno) para asegurar que todas las secuencias dentro del lote tengan la misma longitud.

def collate_fn(batch):
    # 'batch' es una lista de tuplas que contienen las oraciones en inglés y español (en índices).
    # Cada elemento en 'batch' es una tupla con una secuencia en inglés y una secuencia en español.
    
    # 'eng_batch' y 'spa_batch' contienen las secuencias de inglés y español respectivamente.
    # Usamos zip(*batch) para separar las oraciones de inglés y español en dos listas separadas.
    eng_batch, spa_batch = zip(*batch)

    # Para cada secuencia en inglés, recortamos a 'MAX_SEQ_LEN' y usamos '.clone().detach()' para evitar que los cambios
    # en el tensor afecten al original. Esto también asegura que el tensor es independiente de cualquier gráfico de computación.
    eng_batch = [seq[:MAX_SEQ_LEN].clone().detach() for seq in eng_batch]
    
    # Para cada secuencia en español, también la recortamos a 'MAX_SEQ_LEN' y la clonamos para desvincularla.
    spa_batch = [seq[:MAX_SEQ_LEN].clone().detach() for seq in spa_batch]
    
    # Usamos 'torch.nn.utils.rnn.pad_sequence' para rellenar las secuencias en inglés con ceros (padding_value=0) 
    # hasta la longitud máxima en el lote. 'batch_first=True' asegura que la dimensión del batch sea la primera dimensión.
    eng_batch = torch.nn.utils.rnn.pad_sequence(eng_batch, batch_first=True, padding_value=0)
    
    # Hacemos lo mismo para las secuencias en español, asegurando que tengan el mismo tamaño en el lote.
    spa_batch = torch.nn.utils.rnn.pad_sequence(spa_batch, batch_first=True, padding_value=0)
    
    # Retornamos el lote de secuencias en inglés y español después de haber realizado el padding.
    return eng_batch, spa_batch


In [94]:
# Definimos la función 'train', que se encarga de entrenar el modelo durante un número determinado de épocas.
# El entrenamiento se realiza usando el optimizador y la función de pérdida proporcionados.
# Durante cada época, el modelo se ajusta a los datos de entrenamiento en función de la pérdida y el gradiente calculados.
# La función recibe los siguientes parámetros:
# - 'model': El modelo que estamos entrenando.
# - 'dataloader': El DataLoader que proporciona los lotes de datos (eng_batch, spa_batch).
# - 'loss_function': La función de pérdida que se utilizará para calcular el error entre la predicción del modelo y la salida esperada.
# - 'optimiser': El optimizador que ajustará los parámetros del modelo.
# - 'epochs': El número de épocas durante las cuales el modelo será entrenado.

def train(model, dataloader, loss_function, optimiser, epochs):
    # Configuramos el modelo en modo de entrenamiento.
    # 'model.train()' indica que el modelo se encuentra en el estado de entrenamiento, activando comportamientos como el dropout.
    model.train()
    
    # Recorremos las épocas del entrenamiento.
    for epoch in range(epochs):
        # Inicializamos una variable para acumular la pérdida total de la época.
        total_loss = 0 
        
        # Iteramos sobre los lotes del DataLoader.
        # En cada iteración, 'eng_batch' es el lote de secuencias en inglés y 'spa_batch' es el lote de secuencias en español.
        for i, (eng_batch, spa_batch) in enumerate(dataloader):
            # Movemos los lotes al dispositivo (CPU o GPU) donde se encuentra el modelo.
            eng_batch = eng_batch.to(device)
            spa_batch = spa_batch.to(device)
            
            # Preprocesamiento del decoder:
            # Separamos el lote de salida del español en dos partes:
            # - 'target_input' es la entrada al decoder (todas las palabras excepto el último token de cada secuencia).
            # - 'target_output' es la salida esperada del decoder (todas las palabras excepto el primero de cada secuencia).
            target_input = spa_batch[:, :-1]
            target_output = spa_batch[:, 1:].contiguous().view(-1)
            
            # Ponemos a cero los gradientes acumulados de las iteraciones anteriores para evitar que se acumulen en el siguiente paso.
            optimiser.zero_grad()
            
            # Realizamos una pasada hacia adelante a través del modelo con el batch de inglés y el input del decoder.
            output = model(eng_batch, target_input)
            
            # Aplanamos la salida del modelo para que tenga la forma necesaria para calcular la pérdida.
            output = output.view(-1, output.size(-1))
            
            # Calculamos la pérdida entre la salida del modelo y la salida esperada (target_output).
            loss = loss_function(output, target_output)
            
            # Calculamos los gradientes a partir de la pérdida.
            loss.backward()
            
            # Actualizamos los parámetros del modelo usando el optimizador.
            optimiser.step()
            
            # Sumamos la pérdida de esta iteración a la pérdida total.
            total_loss += loss.item()
        
        # Calculamos la pérdida promedio de la época.
        avg_loss = total_loss / len(dataloader)
        
        # Imprimimos la pérdida promedio por cada época.
        print(f'Epoch: {epoch}/{epochs}, Loss: {avg_loss:.4f}')


In [95]:
# Definimos el tamaño del lote (batch) para el entrenamiento. En este caso, el tamaño del lote es 64.
BATCH_SIZE = 8 #64

# Creamos una instancia del dataset que contiene las frases en inglés y español, junto con sus respectivos diccionarios de índices (eng_word2idx y spa_word2idx).
# 'EngSpaDataset' es una clase que hereda de 'Dataset' y está diseñada para gestionar pares de frases en inglés y español,
# así como la conversión de esas frases en listas de índices de palabras (usando los diccionarios 'eng_word2idx' y 'spa_word2idx').
dataset = EngSpaDataset(eng_sentences, spa_sentences, eng_word2idx, spa_word2idx)

# Creamos un DataLoader, que es una herramienta que se encarga de cargar los datos en lotes para el entrenamiento.
# 'DataLoader' toma como entrada el dataset, el tamaño del lote (BATCH_SIZE), si se deben barajar los datos (shuffle=True),
# y una función personalizada de 'collate_fn' que prepara los lotes para el entrenamiento.
# El DataLoader permite iterar sobre el dataset en pequeños lotes, facilitando el entrenamiento del modelo.
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)


In [96]:
# Creamos una instancia del modelo Transformer utilizando los hiperparámetros especificados.

# 'd_model=512': Establece la dimensionalidad de los vectores de características en el modelo Transformer. 
# Esto significa que cada token será representado por un vector de 512 dimensiones, lo que ayuda al modelo a capturar relaciones complejas entre las palabras. 
# Un valor comúnmente usado es 512, que proporciona un buen equilibrio entre poder de representación y eficiencia computacional.

# 'num_heads=8': Define el número de "cabezas" en la atención multi-cabeza (multi-head attention). 
# Cada cabeza de atención aprende una representación diferente de cada token, permitiendo al modelo capturar distintas relaciones y patrones. 
# En este caso, el modelo tendrá 8 cabezas de atención, lo que mejora su capacidad de modelado.

# 'd_ff=2048': Establece el tamaño de la capa de alimentación directa (feed-forward layer). 
# Esta capa toma las salidas de la atención multi-cabeza y las transforma. El valor de 2048 significa que la capa tendrá 2048 neuronas, lo que aumenta la capacidad del modelo para aprender transformaciones complejas.

# 'num_layers=6': Define el número de capas tanto en el encoder como en el decoder. 
# Esto indica cuántas veces se aplicará la atención multi-cabeza y la red de alimentación directa en cada uno de estos componentes. 
# Con 6 capas, el modelo tendrá una capacidad mayor de capturar patrones complejos, aunque también aumenta el costo computacional.

# 'input_vocab_size=eng_vocab_size': Especifica el tamaño del vocabulario de entrada, que corresponde al número de palabras únicas en las frases en inglés. 
# 'eng_vocab_size' es el tamaño del vocabulario de las frases en inglés, y este valor se utiliza para definir la capa de embedding de entrada del modelo.

# 'target_vocab_size=spa_vocab_size': Define el tamaño del vocabulario de salida, correspondiente al número de palabras únicas en las frases en español. 
# 'spa_vocab_size' es el tamaño del vocabulario de las frases en español, y se usa para la capa de embedding en el decoder, así como para la capa de salida del modelo.

# 'max_len=MAX_SEQ_LEN': Establece la longitud máxima de las secuencias que el modelo puede procesar. 
# 'MAX_SEQ_LEN' es una constante que define cuántos tokens (palabras o subpalabras) como máximo el modelo aceptará en las secuencias de entrada y salida. 
# Esto asegura que el modelo pueda manejar secuencias de longitud variable sin quedar limitado.

# 'dropout=0.1': Define la tasa de dropout que se aplica para la regularización durante el entrenamiento. 
# En este caso, el valor de 0.1 significa que el modelo ignorará aleatoriamente el 10% de las conexiones durante el entrenamiento para prevenir el sobreajuste (overfitting).

model = Transformer(d_model=128, num_heads=4, d_ff=512, num_layers=2, #d_model=512  d_ff=2048 num_layer=6
                    input_vocab_size=eng_vocab_size, target_vocab_size=spa_vocab_size,
                    max_len=MAX_SEQ_LEN, dropout=0.1)


In [97]:
# Movemos el modelo al dispositivo especificado (CPU o GPU). Esto asegura que las operaciones del modelo se realicen en el dispositivo que se haya configurado previamente (por ejemplo, 'cuda' si hay una GPU disponible o 'cpu' si no).
# El dispositivo se define antes en el código y se usa para asegurar que tanto el modelo como los datos estén en el mismo lugar para ser procesados.

model = model.to(device)

# Definimos la función de pérdida (loss function). 
# En este caso, utilizamos 'CrossEntropyLoss', que es adecuada para tareas de clasificación múltiple como la traducción de secuencias.
# 'ignore_index=0' le indica a la función de pérdida que ignore las posiciones de los tokens de padding, es decir, los índices con valor 0.
# Esto es importante porque el padding no debe afectar al cálculo de la pérdida, ya que no corresponde a una palabra o token real.

loss_function = nn.CrossEntropyLoss(ignore_index=0)

# Definimos el optimizador que se usará durante el entrenamiento. En este caso, usamos el optimizador 'Adam', que es un optimizador muy popular y eficiente.
# 'model.parameters()' obtiene todos los parámetros del modelo que se actualizarán durante el entrenamiento.
# 'lr=0.0001' especifica la tasa de aprendizaje (learning rate), que controla qué tan rápido el optimizador ajusta los parámetros durante el entrenamiento. 
# En este caso, la tasa de aprendizaje es pequeña para hacer ajustes finos en los parámetros durante el entrenamiento.

optimiser = optim.Adam(model.parameters(), lr=0.0001)


In [100]:
# Llamamos a la función 'train' para entrenar el modelo. El entrenamiento se realiza utilizando los siguientes parámetros:
# - model: El modelo Transformer previamente definido que se entrenará.
# - dataloader: El DataLoader que proporciona los datos en mini-lotes (batches) durante el entrenamiento.
# - loss_function: La función de pérdida que se utilizará para calcular la diferencia entre las predicciones del modelo y las etiquetas verdaderas.
# - optimiser: El optimizador que actualiza los parámetros del modelo en función de la retropropagación (backpropagation).
# - epochs: El número de épocas de entrenamiento. En este caso, entrenaremos el modelo durante 10 épocas.
# Durante cada época, el modelo será alimentado con los datos de entrada, y se calcularán y actualizarán los parámetros del modelo en base a la pérdida.

train(model, dataloader, loss_function, optimiser, epochs=2)



Epoch 1/2
Batch 0/33266 - Loss: 10.9242
Batch 100/33266 - Loss: 9.0613
Batch 200/33266 - Loss: 7.3750
Batch 300/33266 - Loss: 7.1300
Batch 400/33266 - Loss: 7.0763
Batch 500/33266 - Loss: 6.1902
Batch 600/33266 - Loss: 5.9182
Batch 700/33266 - Loss: 7.1199
Batch 800/33266 - Loss: 6.4075
Batch 900/33266 - Loss: 6.2202
Batch 1000/33266 - Loss: 6.1031
Batch 1100/33266 - Loss: 6.1848
Batch 1200/33266 - Loss: 5.7866
Batch 1300/33266 - Loss: 6.1444
Batch 1400/33266 - Loss: 5.4407
Batch 1500/33266 - Loss: 5.0448
Batch 1600/33266 - Loss: 6.5918
Batch 1700/33266 - Loss: 5.5889
Batch 1800/33266 - Loss: 5.7957
Batch 1900/33266 - Loss: 5.4905
Batch 2000/33266 - Loss: 6.2380
Batch 2100/33266 - Loss: 6.4696
Batch 2200/33266 - Loss: 6.1172
Batch 2300/33266 - Loss: 4.9554
Batch 2400/33266 - Loss: 5.5033
Batch 2500/33266 - Loss: 5.2688
Batch 2600/33266 - Loss: 5.8008
Batch 2700/33266 - Loss: 5.4825
Batch 2800/33266 - Loss: 6.4351
Batch 2900/33266 - Loss: 5.6467
Batch 3000/33266 - Loss: 5.2075
Batch 310

In [101]:
# Esta función convierte una oración (sentence) en una lista de índices usando el diccionario 'word2idx'.
# La función toma cada palabra en la oración, la busca en el diccionario 'word2idx', y devuelve su índice correspondiente.
# Si una palabra no está en el diccionario, se utiliza el índice de la palabra desconocida ('<unk>').
def sentence_to_indices(sentence, word2idx):
    return [word2idx.get(word, word2idx['<unk>']) for word in sentence.split()]

# Esta función convierte una lista de índices (indices) en una oración (sentence) usando el diccionario 'idx2word'.
# La función toma cada índice en la lista y lo busca en 'idx2word' para obtener la palabra correspondiente.
# Los índices que corresponden al token de relleno ('<pad>') se ignoran.
def indices_to_sentence(indices, idx2word):
    return ' '.join([idx2word[idx] for idx in indices if idx in idx2word and idx2word[idx] != '<pad>'])

# Esta función realiza la traducción de una oración de inglés a español utilizando el modelo Transformer.
# La función toma como entrada la oración en inglés, las palabras en inglés y español mapeadas a índices ('eng_word2idx' y 'spa_idx2word'),
# y produce la traducción en español.
# La traducción se realiza en modo de evaluación (sin entrenamiento) y sigue los siguientes pasos:
# 1. Preprocesar la oración de entrada en inglés.
# 2. Convertir la oración de inglés en una lista de índices (sentence_to_indices).
# 3. Inicializar el tensor de la entrada y el tensor de destino con el token <sos> (Start of Sentence).
# 4. Realizar una inferencia paso a paso para generar cada palabra en la traducción, deteniéndose cuando se alcanza el token <eos> (End of Sentence).
# 5. Convertir los índices generados a palabras en español utilizando el diccionario 'spa_idx2word'.
def translate_sentence(model, sentence, eng_word2idx, spa_idx2word, max_len=MAX_SEQ_LEN, device='cpu'):
    model.eval()  # Ponemos el modelo en modo de evaluación (sin actualizaciones de parámetros).
    
    # Preprocesamos la oración de entrada (minúsculas, eliminación de espacios innecesarios, etc.).
    sentence = preprocess_sentence(sentence)
    
    # Convertimos la oración de entrada en índices utilizando el diccionario de inglés.
    input_indices = sentence_to_indices(sentence, eng_word2idx)
    
    # Convertimos los índices a un tensor y lo movemos al dispositivo (CPU o GPU).
    input_tensor = torch.tensor(input_indices).unsqueeze(0).to(device)

    # Inicializamos el tensor de destino con el token de inicio de oración (<sos>).
    tgt_indices = [spa_word2idx['<sos>']]
    tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(device)

    with torch.no_grad():  # Desactivamos el cálculo de gradientes, ya que estamos en modo de inferencia.
        # Generamos la traducción palabra por palabra, hasta alcanzar el máximo de longitud o el token <eos>.
        for _ in range(max_len):
            # Ejecutamos el modelo para predecir la siguiente palabra en la secuencia.
            output = model(input_tensor, tgt_tensor)
            
            # Eliminamos la dimensión adicional de la salida y obtenemos la predicción para el siguiente token.
            output = output.squeeze(0)
            
            # Elegimos el token con la probabilidad más alta (máximo valor en la última dimensión).
            next_token = output.argmax(dim=-1)[-1].item()
            
            # Añadimos el siguiente token al tensor de destino.
            tgt_indices.append(next_token)
            
            # Actualizamos el tensor de destino con la secuencia generada hasta ahora.
            tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(device)
            
            # Si el token de salida es <eos>, terminamos la traducción.
            if next_token == spa_word2idx['<eos>']:
                break

    # Convertimos los índices generados a palabras en español y devolvemos la traducción como una cadena de texto.
    return indices_to_sentence(tgt_indices, spa_idx2word)


In [102]:
# Esta función evalúa las traducciones de un conjunto de oraciones de prueba utilizando el modelo Transformer.
# Toma un modelo entrenado, un conjunto de oraciones en inglés, y los diccionarios que mapean palabras a índices para inglés y español.
# La función imprime las traducciones generadas para cada oración de prueba.
# Para cada oración:
# 1. Se traduce utilizando la función 'translate_sentence'.
# 2. Se imprime la oración original (en inglés) y la traducción generada (en español).
def evaluate_translations(model, sentences, eng_word2idx, spa_idx2word, max_len=MAX_SEQ_LEN, device='cpu'):
    for sentence in sentences:
        # Se traduce cada oración del conjunto de oraciones de prueba utilizando la función 'translate_sentence'.
        translation = translate_sentence(model, sentence, eng_word2idx, spa_idx2word, max_len, device)
        
        # Imprime la oración original en inglés y su traducción en español.
        print(f'Input sentence: {sentence}')
        print(f'Traducción: {translation}')
        print()

# Estas son algunas oraciones de ejemplo para evaluar el modelo.
test_sentences = [
    "Hello, how are you?",  # Ejemplo 1
    "I am learning artificial intelligence.",  # Ejemplo 2
    "Artificial intelligence is great.",  # Ejemplo 3
    "Good night!"  # Ejemplo 4
]

# Se asume que el modelo ha sido entrenado y cargado previamente.
# Definimos el dispositivo de ejecución del modelo, que puede ser 'cuda' si hay una GPU disponible, o 'cpu' si no la hay.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mover el modelo al dispositivo adecuado (CPU o GPU).
model = model.to(device)

In [103]:
# Evaluate translations
evaluate_translations(model, test_sentences, eng_word2idx, spa_idx2word, max_len=MAX_SEQ_LEN, device=device)

Input sentence: Hello, how are you?
Traducción: <sos> como estas hola <eos>

Input sentence: I am learning artificial intelligence.
Traducción: <sos> estoy aprendiendo inteligencia inteligencia inteligencia <eos>

Input sentence: Artificial intelligence is great.
Traducción: <sos> la inteligencia es muy grande <eos>

Input sentence: Good night!
Traducción: <sos> buenas noches <eos>

